In [1]:
import pandas as pd

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
df_init = pd.read_csv('../../data/airline-twitter-sentiment/Tweets.csv',encoding='utf-8')

In [4]:
df_init.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
df = df_init[['airline_sentiment','text']]

In [6]:
df.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [7]:
from nltk import word_tokenize 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from string import punctuation

In [18]:
clean_stemmer = PorterStemmer()
# clean_tknzr = TweetTokenizer(preserve_case=False, strip_handles=True)
# clean_stop_words = stopwords.words('english')d(clean_stemmer.stem(item))
#     return stemmed

def clean_tokenize( text):
        tokens = word_tokenize(text)
        return tokens
    
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

def strip_urls(s):
    return re.sub(r'^https?:\/\/.*[\r\n]*', '', s, flags=re.MULTILINE)

def clean_topic(text_list):
    cleaned_text_list =[]
    for text in text_list:
        clean_sentence = []
        tokens = word_tokenize(unicode(text))
        # filtered_words = [w for w in tokens if not w in self.stop_words]
        for word in tokens:
#             word = strip_urls(word)
            word = strip_punctuation(word)
            # if len(word) > 1:
            clean_sentence.append(word)
        cleaned_text_list.append(' '.join(clean_sentence))
    return cleaned_text_list

In [15]:
text_list = df.text.astype(list)

In [16]:
text_list[:10]

0                  @VirginAmerica What @dhepburn said.
1    @VirginAmerica plus you've added commercials t...
2    @VirginAmerica I didn't today... Must mean I n...
3    @VirginAmerica it's really aggressive to blast...
4    @VirginAmerica and it's a really big bad thing...
5    @VirginAmerica seriously would pay $30 a fligh...
6    @VirginAmerica yes, nearly every time I fly VX...
7    @VirginAmerica Really missed a prime opportuni...
8      @virginamerica Well, I didn't…but NOW I DO! :-D
9    @VirginAmerica it was amazing, and arrived an ...
Name: text, dtype: object

In [19]:
cleaned_text = clean_topic(text_list)

In [20]:
cleaned_text[:10]

[u' VirginAmerica What  dhepburn said ',
 u' VirginAmerica plus you ve added commercials to the experience  tacky ',
 u' VirginAmerica I did nt today  Must mean I need to take another trip ',
 u' VirginAmerica it s really aggressive to blast obnoxious  entertainment  in your guests  faces  amp  they have little recourse',
 u' VirginAmerica and it s a really big bad thing about it',
 u' VirginAmerica seriously would pay  30 a flight for seats that did nt have this playing  it s really the only bad thing about flying VA',
 u' VirginAmerica yes  nearly every time I fly VX this \u201cear worm\u201d won\u2019t go away  ',
 u' VirginAmerica Really missed a prime opportunity for Men Without Hats parody  there  https  tcomWpG7grEZP',
 u' virginamerica Well  I didnt\u2026but NOW I DO   D',
 u' VirginAmerica it was amazing  and arrived an hour early  You re too good to me ']

In [21]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2)
tf = tf_vectorizer.fit_transform(cleaned_text)

In [22]:
tf[0]

<1x6487 sparse matrix of type '<type 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn import neighbors
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score

In [25]:
def k_fold_classifier(classifier, data, labels, n_folds=5):
    k_fold = KFold(n=data.shape[0], n_folds=n_folds)
    scores = []
    for train_indices, test_indices in k_fold:

        train_text = data[train_indices]
        train_y = labels[train_indices]

        test_text = data[test_indices]
        test_y = labels[test_indices]

        classifier.fit(train_text, train_y)
        predictions = classifier.predict(test_text)
        score = accuracy_score(test_y,predictions)
        scores.append(score)


    print'Average Accuracy Score:', sum(scores)/len(scores)

In [26]:
clf = neighbors.KNeighborsClassifier(1)
k_fold_classifier(clf,tf,df.airline_sentiment)

Average Accuracy Score: 0.478278688525


In [31]:
clf.kneighbors(tf[2],2)

(array([[ 0.        ,  3.16227766]]), array([[  2, 241]]))

In [ ]:
df.iloc[241]

In [ ]:
df.iloc[3]

In [32]:
for k in range(1,4):
    clf = neighbors.KNeighborsClassifier(k,metric='euclidean')
    k_fold_classifier(clf,tf,df.airline_sentiment)

Average Accuracy Score: 0.478278688525
Average Accuracy Score: 0.556489071038
Average Accuracy Score: 0.50806010929


In [33]:
for k in range(1,5):
    clf = neighbors.KNeighborsClassifier(k,weight='distance')
    k_fold_classifier(clf,tf,df.airline_sentiment)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Passing additional arguments to the metric function as **kwargs is deprecated and will no longer be supported in 0.18. Use metric_params instead.
  from ipykernel import kernelapp as app


Average Accuracy Score: 0.478278688525
Average Accuracy Score: 0.556489071038
Average Accuracy Score: 0.50806010929
Average Accuracy Score: 0.50143442623
